# Time Series SVR Algorithm

This algorithm is used to forecast FOREX prices for the EURUSD pair. It fetches data from MetaTrader5 and applies feature engineering to create additional data.

The forecasted values are set to:
- **Position 1** if the prediction indicates that the price will go up in the next hour
- **Position 0** if the prediction indicates that the price will go down in the nex hourx hour

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-darkgrid")
from matplotlib import cycler
import warnings
warnings.filterwarnings("ignore")
import ta
import seaborn as sns
import MetaTrader5 as mt5
from datetime import datetime
from UsefulFunctions import data, backtest

In [141]:
# Initiate bound between MetaTrader5 and Python
mt5.initialize()

True

In [143]:
# Fetching data from MetaTrader5 from 2015 to the present using the custom function "get_rates" from the data.py file
# The historical data retrieved is of 1-hour timeframe
df = data.get_rates("EURUSD", mt5.TIMEFRAME_H1, datetime(2015,1,1))
df.head()

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2015-01-02 08:00:00,1.2051,1.2057,1.2048,1.2054,134,20,0
2015-01-02 09:00:00,1.2053,1.2058,1.2034,1.2055,650,20,0
2015-01-02 10:00:00,1.2056,1.2069,1.2044,1.2047,871,20,0
2015-01-02 11:00:00,1.2048,1.2060,1.2038,1.2057,875,20,0
2015-01-02 12:00:00,1.2056,1.2056,1.2042,1.2050,441,20,0


In [145]:
# Drop columns spread and real_volume and remane columns tick_volumne to volumne
df = df[["open", "high", "low", "close", "tick_volume"]]
df.rename(columns={'tick_volume': 'volume'}, inplace=True)
df.head()

,open,high,low,close,volume
time,,,,,
2015-01-02 08:00:00,1.2051,1.2057,1.2048,1.2054,134
2015-01-02 09:00:00,1.2053,1.2058,1.2034,1.2055,650
2015-01-02 10:00:00,1.2056,1.2069,1.2044,1.2047,871
2015-01-02 11:00:00,1.2048,1.2060,1.2038,1.2057,875
2015-01-02 12:00:00,1.2056,1.2056,1.2042,1.2050,441


## Feature engineering 
**Creating new input variables from raw data**

In [148]:
# Create a second DataFrame from the close column and compute the percent change in a new column
df_copy = df[["close"]]
df_copy["pct_change"] = df_copy["close"].pct_change(1)
df_copy.head()

,close,pct_change
time,,
2015-01-02 08:00:00,1.2054,NaN
2015-01-02 09:00:00,1.2055,0.000083
2015-01-02 10:00:00,1.2047,-0.000664
2015-01-02 11:00:00,1.2057,0.000830
2015-01-02 12:00:00,1.2050,-0.000581


In [150]:
# New DataFrame with all indications from "ta" library and shifting them by one row
df_indicators = ta.add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume", fillna=True).shift(1)
df_indicators.head()

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02 09:00:00,1.2051,1.2057,1.2048,1.2054,134.0,44.666667,1.340000e+02,0.333333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.205400,0.000000,0.000000,0.000000
2015-01-02 10:00:00,1.2053,1.2058,1.2034,1.2055,650.0,532.166667,7.840000e+02,0.678784,0.065000,-0.240000,...,0.000662,0.000132,0.000529,23.900744,4.780149,19.120596,1.205444,0.008296,0.008296,0.008296
2015-01-02 11:00:00,1.2056,1.2069,1.2044,1.2047,871.0,-129.793333,1.844674e+19,-0.078425,-0.043829,0.301378,...,-0.004122,-0.000718,-0.003403,40.437201,11.911559,28.525642,1.205129,-0.066363,-0.066385,-0.058072
2015-01-02 12:00:00,1.2048,1.2060,1.2038,1.2057,875.0,506.570303,7.880000e+02,0.200225,0.087433,-0.188571,...,-0.001204,-0.000816,-0.000389,47.232938,18.975835,28.257103,1.205374,0.083008,0.082974,0.024888


In [164]:
# Here I concatenated both dataframes and drop the close column (which wasn't shifted) from df_copy
df_to_work = pd.concat((df_indicators, df_copy["pct_change"]), axis=1)
df_to_work.dropna(inplace=True)
df_to_work.head()

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,pct_change
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 09:00:00,1.2051,1.2057,1.2048,1.2054,134.0,44.666667,1.340000e+02,0.333333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.205400,0.000000,0.000000,0.000000,0.000083
2015-01-02 10:00:00,1.2053,1.2058,1.2034,1.2055,650.0,532.166667,7.840000e+02,0.678784,0.065000,-0.240000,...,0.000132,0.000529,23.900744,4.780149,19.120596,1.205444,0.008296,0.008296,0.008296,-0.000664
2015-01-02 11:00:00,1.2056,1.2069,1.2044,1.2047,871.0,-129.793333,1.844674e+19,-0.078425,-0.043829,0.301378,...,-0.000718,-0.003403,40.437201,11.911559,28.525642,1.205129,-0.066363,-0.066385,-0.058072,0.000830
2015-01-02 12:00:00,1.2048,1.2060,1.2038,1.2057,875.0,506.570303,7.880000e+02,0.200225,0.087433,-0.188571,...,-0.000816,-0.000389,47.232938,18.975835,28.257103,1.205374,0.083008,0.082974,0.024888,-0.000581
2015-01-02 13:00:00,1.2056,1.2056,1.2042,1.2050,441.0,569.570303,3.470000e+02,0.191710,0.030842,0.000000,...,-0.001360,-0.002178,42.940551,23.768778,19.171773,1.205216,-0.058058,-0.058074,-0.033184,0.000249
